<a href="https://colab.research.google.com/github/Sergio-ddf/emit-llm/blob/main/emit_setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EMit Emotion Detection Task

# 1. Setup e import


In [1]:
# API Key WandB
from google.colab import userdata
import os, wandb

os.environ["WANDB_API_KEY"] = userdata.get('WANDB_KEY')
wandb.login()

wandb: Currently logged in as: ddfsergio9 (ddfsergio9-sapienza-universit-di-roma) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [2]:
# Installazione delle dipendenze
!pip install -q datasets emoji iterative-stratification evaluate tokenizers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is 

In [10]:
# Import principali e informazioni sull'ambiente
import os
import re

import numpy as np
import pandas as pd
import torch
import emoji

from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, classification_report
from sklearn.multiclass import OneVsRestClassifier

from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer
)
from torch.nn import BCEWithLogitsLoss
from torch.nn.functional import sigmoid

print(f"PyTorch    : {torch.__version__} (CUDA disponibile: {torch.cuda.is_available()})")

PyTorch    : 2.6.0+cu124 (CUDA disponibile: False)


# 2. Configurazione e percorsi


In [6]:
DATA_DIR     = ""  # path della cartella con i dati
MODEL_NAME   = "Musixmatch/umberto-commoncrawl-cased-v1"
DEVICE       = torch.device("cuda" if torch.cuda.is_available() else "cpu")
LABELS       = ['Anger','Anticipation','Disgust','Fear','Joy',
                'Love','Neutral','Sadness','Surprise','Trust']
NUM_LABELS   = len(LABELS)


# 3. Caricamento dati e statistiche


In [8]:
train_a = pd.read_csv(os.path.join(DATA_DIR, "emit_train_A.csv"))
test_in = pd.read_csv(os.path.join(DATA_DIR, "emit_test.csv"))

print("Train A:", train_a.shape, "Test in-domain:", test_in.shape)
display(train_a.head())

# distribuzione etichette
counts = train_a[LABELS].sum().sort_values(ascending=False)
display(counts.to_frame("etichette"))


Train A: (5966, 12) Test in-domain: (1000, 14)


,id,text,Anger,Anticipation,Disgust,Fear,Joy,Love,Neutral,Sadness,Surprise,Trust
0,04b19fcefed25f6919fbbb8fe6ac4f1e,RT @user: Serve aggiungere altro? #Ulisse INIZ...,0,1,0,0,1,1,0,0,0,0
1,6320e808285b5e1613dbd28580affd21,RT @user: La differenza tra chi ha studiato ed...,0,0,0,0,0,0,1,0,0,0
2,abdddf2c6231ca527bbaf808e8c11085,Metà della sobrietà di #IlariaCucchi dovrebbe ...,0,0,0,0,0,0,0,0,0,1
3,8b6a76b34ed6bdb23d61ba9ef386ecb8,LENÙ PER PIACERE SCAPPA STO IN ANSIA PER TE TI...,0,1,0,1,0,0,0,0,0,0
4,913743b451b8178fa138c7b27f26fac6,@user Nooooo!!! La signora Cecchini 😢 #DonMatt...,0,0,0,0,0,0,0,1,0,0


,etichette
Trust,1665
Neutral,1322
Disgust,874
Joy,650
Love,633
Surprise,591
Anticipation,547
Sadness,545
Anger,367
Fear,91


# 4. Pulizia testo


In [11]:
URL, USER, TAG = "<URL>", "<USER>", "<HASHTAG>"

def clean(text: str) -> str:
    t = re.sub(r'https?://\S+', URL, text)
    t = re.sub(r'@\w+', USER, t)
    t = re.sub(r'#(\w+)', TAG + r' \1', t)
    t = emoji.demojize(text, language='it')
    return t.strip()

train_a['text_clean'] = train_a['text'].map(clean)
train_a[['text', 'text_clean']].head()



,text,text_clean
0,RT @user: Serve aggiungere altro? #Ulisse INIZ...,RT @user: Serve aggiungere altro? #Ulisse INIZ...
1,RT @user: La differenza tra chi ha studiato ed...,RT @user: La differenza tra chi ha studiato ed...
2,Metà della sobrietà di #IlariaCucchi dovrebbe ...,Metà della sobrietà di #IlariaCucchi dovrebbe ...
3,LENÙ PER PIACERE SCAPPA STO IN ANSIA PER TE TI...,LENÙ PER PIACERE SCAPPA STO IN ANSIA PER TE TI...
4,@user Nooooo!!! La signora Cecchini 😢 #DonMatt...,@user Nooooo!!! La signora Cecchini :faccina_c...


# 5. Split stratificato 90/10


In [12]:
X = train_a['text_clean'].values
Y = train_a[LABELS].values

msss = MultilabelStratifiedShuffleSplit(test_size=0.1, random_state=42)
train_idx, val_idx = next(msss.split(X, Y))

train_df = train_a.iloc[train_idx].reset_index(drop=True)
val_df   = train_a.iloc[val_idx].reset_index(drop=True)

print("Train/Val:", train_df.shape[0], "/", val_df.shape[0])


Train/Val: 5381 / 585


# 6. Baseline TF-IDF + Logistic Regression


In [13]:
vec = TfidfVectorizer(ngram_range=(1,2), max_features=5000)
X_tr = vec.fit_transform(train_df['text_clean'])
X_va = vec.transform(val_df['text_clean'])

y_tr = train_df[LABELS].values
y_va = val_df[LABELS].values

clf = OneVsRestClassifier(LogisticRegression(max_iter=1000, random_state=42))
clf.fit(X_tr, y_tr)
y_pr = clf.predict(X_va)

print("TF-IDF+LR macro-F1:", f1_score(y_va, y_pr, average='macro'))
print(classification_report(y_va, y_pr, target_names=LABELS, zero_division=0))


TF-IDF+LR macro-F1: 0.15714584225470465
              precision    recall  f1-score   support

       Anger       0.00      0.00      0.00        37
Anticipation       0.71      0.09      0.16        55
     Disgust       1.00      0.01      0.02        87
        Fear       0.00      0.00      0.00         9
         Joy       0.80      0.12      0.21        65
        Love       0.72      0.21      0.32        63
     Neutral       0.60      0.11      0.19       132
     Sadness       1.00      0.09      0.17        55
    Surprise       0.80      0.07      0.12        59
       Trust       0.82      0.24      0.37       167

   micro avg       0.76      0.12      0.21       729
   macro avg       0.65      0.09      0.16       729
weighted avg       0.74      0.12      0.20       729
 samples avg       0.15      0.13      0.14       729



# 7. Preparazione HF Dataset


In [14]:
# colonne labels in float vector
for df in (train_df, val_df):
    df['labels'] = df[LABELS].astype(float).values.tolist()

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
MAX_LEN   = 128

def tokenize_fn(batch):
    enc = tokenizer(batch['text_clean'],
                    padding='max_length',
                    truncation=True,
                    max_length=MAX_LEN)
    enc['labels'] = batch['labels']
    return enc

drop_cols = ['text', 'text_clean'] + LABELS

dtrain = Dataset.from_pandas(train_df)\
           .map(tokenize_fn, batched=True, remove_columns=drop_cols)\
           .with_format('torch', columns=['input_ids','attention_mask','labels'])
dval   = Dataset.from_pandas(val_df)\
           .map(tokenize_fn, batched=True, remove_columns=drop_cols)\
           .with_format('torch', columns=['input_ids','attention_mask','labels'])


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/508 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/794k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.68M [00:00<?, ?B/s]

Map:   0%|          | 0/5381 [00:00<?, ? examples/s]

Map:   0%|          | 0/585 [00:00<?, ? examples/s]

# 8. Fine-tuning UmBERTo con BCE pesata


In [15]:
# pesi per BCE
freqs       = train_df[LABELS].mean().values
pos_weights = torch.tensor((1 - freqs) / freqs, device=DEVICE)

class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        inputs.pop("num_items_in_batch", None)
        labels = inputs.pop("labels")
        logits = model(**inputs).logits
        loss   = BCEWithLogitsLoss(pos_weight=pos_weights)(logits, labels.float())
        return (loss, logits) if return_outputs else loss

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=NUM_LABELS,
    problem_type='multi_label_classification'
).to(DEVICE)
model.config.id2label = {i:l for i,l in enumerate(LABELS)}
model.config.label2id = {l:i for i,l in enumerate(LABELS)}

args = TrainingArguments(
    output_dir='ckpt/umberto',
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    learning_rate=2e-5,
    num_train_epochs=10,
    lr_scheduler_type='linear',
    warmup_steps=500,
    logging_steps=50,
    eval_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    metric_for_best_model='eval_macro_f1',
    report_to=None
)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    probs  = sigmoid(torch.tensor(logits))
    preds  = (probs >= 0.5).int().numpy()
    return {"eval_macro_f1": float(f1_score(labels, preds, average='macro', zero_division=0))}

trainer = WeightedTrainer(
    model=model,
    args=args,
    train_dataset=dtrain,
    eval_dataset=dval,
    compute_metrics=compute_metrics
)


pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at Musixmatch/umberto-commoncrawl-cased-v1 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

In [ ]:

trainer.train()


# 9. Ottimizzazione soglie e valutazione finale


In [ ]:
# logits e labels
preds_out = trainer.predict(dval)
val_probs = sigmoid(torch.tensor(preds_out.predictions)).numpy()
val_labels= val_df[LABELS].values.astype(int)

# soglie per classe
best_t = []
for i in range(NUM_LABELS):
    best_f1, best_thresh = 0, 0.5
    for t in np.linspace(0.1, 0.9, 81):
        f1 = f1_score(val_labels[:,i], (val_probs[:,i]>=t).astype(int))
        if f1>best_f1:
            best_f1, best_thresh = f1, t
    best_t.append(best_thresh)

print(dict(zip(LABELS, best_t)))

# macro-F1 ottimizzata
preds_opt = (val_probs >= np.array(best_t)).astype(int)
print("Final optimized macro-F1:", f1_score(val_labels, preds_opt, average='macro', zero_division=0))
